<a href="https://colab.research.google.com/github/SurbhiS19/Hate_Speech_Transformers/blob/main/Sentiment_Analysis_with_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizerFast
from transformers import Trainer, TrainingArguments
from transformers import DistilBertForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.0 MB/s eta 0:00:00


In [ ]:
import torch
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data = pd.read_csv("train_E6oV3lV.csv")

In [ ]:
from sklearn.model_selection import train_test_split

all_data=data[['tweet','label']]
x_train, x_test, y_train, y_test = train_test_split(all_data.tweet,all_data.label, test_size=0.2, random_state=1)

In [ ]:
training_data = pd.DataFrame(x_train)
training_data['label'] = pd.DataFrame(y_train)
testing_data = pd.DataFrame(x_test)
testing_data['label'] = pd.DataFrame(y_test)

In [ ]:
# Converting labels and tweets to list
train_tweets = training_data.tweet.to_list()
train_labels = training_data.label.to_list()

test_tweets = testing_data.tweet.to_list()
test_labels = testing_data.label.to_list()

In [ ]:
#tokenization
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_inputs= tokenizer(train_tweets,padding="max_length",truncation=True)
test_inputs = tokenizer(test_tweets,padding="max_length",truncation=True)
# inputs[1]

In [ ]:
# embeddings

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_inputs,train_labels)
test_dataset = IMDbDataset(test_inputs,test_labels)

In [ ]:
# print(len(train_dataset.__getitem__(2)))

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)
# model.to(device)

# torch.cuda.is_available()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
training_args = TrainingArguments(output_dir = './results',
                                  num_train_epochs = 1,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=64,
                                  warmup_steps =100,
                                  weight_decay=0.01,
                                  logging_dir='./logs',
                                  logging_steps=10)
trainer = Trainer(model=model,
                  args = training_args,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset,
                  tokenizer=tokenizer,
                  data_collator=data_collator
                  )

trainer.train()

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.650900
20,0.540300
30,0.342500
40,0.357600
50,0.279700
60,0.180400
70,0.306800
80,0.170500
90,0.241600
100,0.220800


In [14]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [15]:
def check_results(test_data):
  input_ids = torch.tensor(test_data["input_ids"]).to(device)
  attention_mask = torch.tensor(test_data["attention_mask"]).to(device)
  with torch.no_grad():
    outputs = model(input_ids.unsqueeze(0),attention_mask.unsqueeze(0))
  y=np.argmax(outputs[0].to('cpu').numpy())

  return y

In [16]:
l2 =[]

for i in testing_data["tweet"]:
  test_data = tokenizer(i,truncation=True, padding=True)
  input_ids = torch.tensor(test_data["input_ids"]).to(device)
  attention_mask = torch.tensor(test_data["attention_mask"]).to(device)
  op = check_results(test_data)

  l2.append(op)


In [34]:
l1= testing_data.label.values.tolist()
accuracy = (np.squeeze(l1)==np.squeeze(l2)).sum()/len(l1)

accuracy

0.9698107304864696